In [ ]:
from ibapi.sync_wrapper import *
import pandas as pd

In [ ]:
app = TWSSyncWrapper(timeout=24 * 60 * 60)
app.connect_and_start(host="127.0.0.1", port=7496, client_id=1)
app.get_current_time()

api_cache = {}
contract_tz_map = {
    'OSE.JPN': 'Japan',
    'IDEALPRO': 'US/Eastern',
    'COMEX': 'US/Eastern',
    'NYMEX': 'US/Eastern',
    'SMART': 'US/Eastern'
}

In [ ]:
# app.disconnect_and_stop()

In [ ]:
def bars2ohlcv(bars, tz):
    tz_map = {
        'US/Eastern': 'US/Eastern',
        'Japan': 'Asia/Tokyo',
    }
    df = pd.DataFrame.from_dict([{'date': bar.date, 'open': bar.open, 'high': bar.high, 'low': bar.low, 'close': bar.close, 'volume': bar.volume} for bar in bars])
    df['date'] = pd.to_datetime(df['date'], format=f'%Y%m%d %H:%M:%S {tz}').dt.tz_localize(tz_map[tz])
    df['date'] = df['date'].dt.tz_convert('UTC')
    df.set_index('date', inplace=True)
    return df


In [ ]:
def merge_df(df1, df2, df3):
    merged_df = pd.merge(df1, df2, left_index=True, right_index=True, how='inner', suffixes=('_1', '_2'))
    merged_df = pd.merge(merged_df, df3, left_index=True, right_index=True, how='inner')
    merged_df.rename(columns={'close_1': 'a0', 'close_2': 'b0', 'close': 'c0'}, inplace=True)
    return merged_df[['a0', 'b0', 'c0']]

In [ ]:
def compare3symbols(symbols, duration, bar_size):
    c1, c2, c3 = symbols
    _, (df1, df2) = compare2symbols((c1, c2), duration, bar_size)
    df3 = get_historical_data(c3, duration, bar_size, "MIDPOINT")
    df = merge_df(df1, df2, df3)
    return df, (df1, df2, df3)

def compare2symbols(symbols, duration, bar_size):
    c1, c2 = symbols

    df1 = get_historical_data(c1, duration, bar_size, "TRADES")
    df2 = get_historical_data(c2, duration, bar_size, "TRADES")

    df = pd.merge(df1, df2, on='date', suffixes=('_1', '_2'), how='inner')
    return df, (df1, df2)

def get_historical_data(contract, duration, bar_size, what_to_show):
    key = f'{contract}_{duration}_{bar_size}'
    if api_cache.get(key) is None:
        api_cache[key] = app.get_historical_data(
            contract=contract, 
            end_date_time="", 
            duration_str=duration,
            bar_size_setting=bar_size,
            what_to_show=what_to_show, 
            use_rth=False,
            timeout=24*60*60)
    return bars2ohlcv(api_cache[key], contract_tz_map.get(contract.exchange))

In [ ]:
USDJPY = Contract()
USDJPY.symbol = "USD"
USDJPY.secType = "CASH"
USDJPY.exchange = "IDEALPRO"
USDJPY.currency = "JPY"

USDCNH = Contract()
USDCNH.symbol = "USD"
USDCNH.secType = "CASH"
USDCNH.exchange = "IDEALPRO"
USDCNH.currency = "CNH"

GD = Contract()
GD.symbol = "GOLD"
GD.secType = "CONTFUT"
GD.exchange = "OSE.JPN"
GD.currency = "JPY"

M_PT = Contract()
M_PT.symbol = "M-PT"
M_PT.secType = "CONTFUT"
M_PT.exchange = "OSE.JPN"
M_PT.currency = "JPY"

GC = Contract()
GC.symbol = "GC"
GC.secType = "CONTFUT"
GC.exchange = "COMEX"
GC.currency = "USD"

MGC = Contract()
MGC.symbol = "MGC"
MGC.secType = "CONTFUT"
MGC.exchange = "COMEX"
MGC.currency = "USD"

_1OZ = Contract()
_1OZ.symbol = "1OZ"
_1OZ.secType = "CONTFUT"
_1OZ.exchange = "COMEX"
_1OZ.currency = "USD"

PL = Contract()
PL.symbol = "PL"
PL.secType = "CONTFUT"
PL.exchange = "NYMEX"
PL.currency = "USD"

PPLT = Contract()
PPLT.symbol = "PPLT"
PPLT.secType = "STK"
PPLT.exchange = "SMART"
PPLT.currency = "USD"

In [ ]:
import numpy as np
df, _ = compare2symbols((PPLT, PL), '6 M', '5 mins')
df['a-b'] = np.log(df['close_2']) - np.log(df['close_1'])


In [ ]:
df, (df1, df2, df3) = compare3symbols((M_PT, PL, USDJPY), '2 M', '1 min')

df['a'] = df['a0'] / df['c0'] * 31.1034768
df['b'] = df['b0']
df['a-b'] = df['a'] - df['b']

In [ ]:
import matplotlib.pyplot as plt

lower, upper = df['a-b'].quantile([0.025, 0.975])
print(f'relation: {df['a'].corr(df['b'])}')
print(f'95%: {lower:.4f}, {upper:.4f}')
print(f'N ({df['a-b'].mean():.4f}, {df['a-b'].std():.4f})')

plt.figure(figsize=(10, 4))
plt.scatter(df.index, df['a-b'], s=1)
plt.axhline(y=lower, color='r', linestyle='--')
plt.axhline(y=upper, color='r', linestyle='--')
plt.show()


In [ ]:
df = get_historical_data(PL, '1 Y', '1 min', 'TRADES')
df.to_csv('data/PL-1min.csv')

In [ ]:
df = get_historical_data(M_PT, '1 Y', '1 min', 'TRADES')
df.to_csv('data/M-PT-1min.csv')
df = get_historical_data(USDJPY, '1 Y', '1 min', "MIDPOINT")
df.to_csv('data/USDJPY-1min.csv')

PLAT = Contract()
PLAT.symbol = "PLAT"
PLAT.secType = "CONTFUT"
PLAT.exchange = "OSE.JPN"
PLAT.currency = "JPY"

df = get_historical_data(PLAT, '1 Y', '1 min', "TRADES")
df.to_csv('data/PLAT-1min.csv')